# Tutorial 1. Cost and scaling model (for a turbine)

WISDEM offers different levels of fidelity for estimating mass, cost, and LCOE. The simplest form is an implementation of a "spreadsheet" type model, where given a few high-level parameters, such as `rotor diameter` and `machine_rating`, the user can use a series of empirical relationships (regression-based) to estimate turbine mass and cost. This is called the Cost and Scaling Model and we will call it here.

First, import the WISDEM code that will model our costs. Also import the pieces of OpenMDAO that we'll need for the calculation.

In [1]:
from wisdem.turbine_costsse.nrel_csm_tcc_2015 import nrel_csm_2015
from openmdao.api import Problem
import numpy as np

## Cost and Mass model
`nrel_csm_2015` is a class that contains a model to estimate the masses and costs of turbine components.

First, we instantiate the `nrel_csm_2015` class. Then we make an OpenMDAO `Problem` that uses this class. After that, we set the inputs for our model.

In [2]:
turbine = nrel_csm_2015()
prob = Problem(turbine)
prob.setup()

prob['rotor_diameter'] = 126.0
prob['turbine_class'] = 1
prob['blade_has_carbon'] = False
prob['blade_number'] = 3    
prob['machine_rating'] = 5000.0
prob['hub_height'] = 90.0
prob['bearing_number'] = 2
prob['crane'] = True
prob['max_tip_speed'] = 80.0
prob['max_efficiency'] = 0.90

Now we are set to run the model:

In [3]:
prob.run_driver()

False

That's it!  The model has been executed and now we have to get at the outputs we are interested in.  Since we don't know the names of the outputs variables, we can just exhausively list (and store) all of the model inputs and outputs. There are simple commands for doing so:

In [4]:
myinputs = prob.model.list_inputs(units=True)

219 Input(s) in 'model'
-----------------------

varname                                      value               units 
-------------------------------------------  ------------------  ------
top
  nrel_csm_mass
    blade
      rotor_diameter                         [126.]              m     
      blade_mass_coeff                       [0.5]               None  
      blade_user_exp                         [2.5]               None  
    hub
      blade_mass                             [18590.66820649]    kg    
      hub_mass_coeff                         [2.3]               None  
      hub_mass_intercept                     [1320.]             None  
    pitch
      blade_mass                             [18590.66820649]    kg    
      pitch_bearing_mass_coeff               [0.1295]            None  
      pitch_bearing_mass_intercept           [491.31]            None  
      bearing_housing_percent                [0.328]             None  
      mass_sys_offset                  

Now list the model outputs.

In [5]:
myoutputs = prob.model.list_outputs(units=True)

91 Explicit Output(s) in 'model'
--------------------------------

varname                                      value               units 
-------------------------------------------  ------------------  ------
top
  nrel_csm_mass
    sharedIndeps
      machine_rating                         [5000.]             kW    
      rotor_diameter                         [126.]              m     
    blade
      blade_mass                             [18590.66820649]    kg    
    hub
      hub_mass                               [44078.53687493]    kg    
    pitch
      pitch_system_mass                      [10798.90594644]    kg    
    spinner
      spinner_mass                           [973.]              kg    
    lss
      lss_mass                               [22820.27928238]    kg    
    bearing
      main_bearing_mass                      [2245.41649102]     kg    
    torque
      rotor_torque                           [4375000.]          N*m   
    gearbox
      gearbox_mass   

## User Overrides
The Cost and Scaling model can be used to estimate mass and cost from a limited set of inputs.  If the user already knows the mass or cost of a particular component, the easiest thing to do is to override the mass or cost scaling coefficient. This can also be used to conduct scaling studies at different and mass and cost growth sensitivities:

In [7]:
prob['gearbox_mass_coeff'] = 75.0
prob['gearbox_mass_cost_coeff'] = 10.0
prob.run_driver()

False

If we store these inputs and outputs into new containers, we can easily compare the impact of the changes

In [8]:
newinputs = prob.model.list_inputs(units=True)

219 Input(s) in 'model'
-----------------------

varname                                      value               units 
-------------------------------------------  ------------------  ------
top
  nrel_csm_mass
    blade
      rotor_diameter                         [126.]              m     
      blade_mass_coeff                       [0.5]               None  
      blade_user_exp                         [2.5]               None  
    hub
      blade_mass                             [18590.66820649]    kg    
      hub_mass_coeff                         [2.3]               None  
      hub_mass_intercept                     [1320.]             None  
    pitch
      blade_mass                             [18590.66820649]    kg    
      pitch_bearing_mass_coeff               [0.1295]            None  
      pitch_bearing_mass_intercept           [491.31]            None  
      bearing_housing_percent                [0.328]             None  
      mass_sys_offset                  

In [9]:
newoutputs = prob.model.list_outputs(units=True)

91 Explicit Output(s) in 'model'
--------------------------------

varname                                      value               units 
-------------------------------------------  ------------------  ------
top
  nrel_csm_mass
    sharedIndeps
      machine_rating                         [5000.]             kW    
      rotor_diameter                         [126.]              m     
    blade
      blade_mass                             [18590.66820649]    kg    
    hub
      hub_mass                               [44078.53687493]    kg    
    pitch
      pitch_system_mass                      [10798.90594644]    kg    
    spinner
      spinner_mass                           [973.]              kg    
    lss
      lss_mass                               [22820.27928238]    kg    
    bearing
      main_bearing_mass                      [2245.41649102]     kg    
    torque
      rotor_torque                           [4375000.]          N*m   
    gearbox
      gearbox_mass   

Scrolling through the outputs, we can see that these new values for gearbox mass and cost changed the cost of the turbine from \\$726/kW to \\$672/kW.

Another approach is to split out the mass scaling and cost scaling routines, but we will leave that for another tutorial